In [ ]:
# Task 3: Correlation Analysis - Interactive Notebook

import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from task3_correlation_analysis import CorrelationAnalyzer

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Task 3: Correlation Analysis between News Sentiment and Stock Returns")